In [17]:
import pandas as pd
import math
import numpy as np
import requests
import time
from collections import Counter
from sklearn.model_selection import train_test_split

In [18]:
df = pd.read_excel('./data.xlsx', index_col=0)
df.columns

Index(['pair', 'token_name', 'token_address', 'price_b', 'price_s',
       'token_age_b', 'token_age_s', 'transfers', 'transactions', 'buys_m5',
       'sells_m5', 'buys_h1', 'sells_h1', 'buys_h6', 'sells_h6', 'buys_h24',
       'sells_h24', 'volume_m5', 'volume_h1', 'volume_h6', 'volume_h24',
       'price_change_m5', 'price_change_h1', 'price_change_h6',
       'price_change_h24', 'liquidity', 'fdv', 'market_cap',
       'is_mutable_metadata', 'is_telegram', 'is_twitter', 'is_website',
       'boosts', 'telegram_members', 'is_telegram_error', 'twitter_days',
       'twitter_tweets', 'twitter_followers', 'twitter_smart_followers',
       'is_twitter_error', 'opening_date', 'closing_date', 'sns_bought',
       'sns_sold', 'sns_held_all', 'sns_sold_some', 'sns_sold_all',
       'tt_bought', 'tt_sold', 'PNL', 'status', 'mode'],
      dtype='object')

In [19]:
drop_columns = ['pair', 'token_name',
    'buys_h6', 'sells_h6', 'buys_h24', 'sells_h24',
    'volume_h6', 'volume_h24', 'fdv',
    'price_b', 'price_s', 'token_age_s', 
    'opening_date', 'closing_date',
    'status', 'mode']
df = df.drop(drop_columns, axis=1)
df.head()

,token_address,token_age_b,transfers,transactions,buys_m5,sells_m5,buys_h1,sells_h1,volume_m5,volume_h1,...,twitter_smart_followers,is_twitter_error,sns_bought,sns_sold,sns_held_all,sns_sold_some,sns_sold_all,tt_bought,tt_sold,PNL
id,,,,,,,,,,,,,,,,,,,,,
5203,BiMacNBZjf9dJxkcXvtScsGQbv4kJAKxJsa8Z7JVj24v,"2,78",NaN,207,135,72,135,72,"153924,19","153924,19",...,NaN,0,24200.0 24000.0 23700.0 7.06 0.1 0.1 0.1 0.23 ...,0 0 0 5.57 0 0 0 0 0 0 0 20.0 0 0 16.0 0 5.65 ...,74.0,4.0,22.0,0 0 0 0 0 0 381.0 0 0 389.0 0 0 282.0 282.0 23...,9624.0 9171.0 8595.0 8138.0 7638.0 7204.0 7176...,"-31,62"
5202,BBbMM5At7dBAi5UQuFnnYXnSZBH4AL7fiNS2eUwTBijJ,"5,03",NaN,1554,908,646,908,646,"489175,41","489175,41",...,NaN,0,697.0 0 233.0 22100.0 2326.0 2328.0 2333.0 234...,0 4011.0 147.0 22100.0 0 1912.0 2805.0 0 0 39....,28.0,12.0,60.0,0 0 0 0 0 0 0 0 0 0 0 3024.0 0 0 0 0 0 0 418.0...,15900.0 5372.0 4371.0 4018.0 4011.0 2974.0 297...,"73,71"
5200,445fDjExGhSq6FG8VAiPNLgkPnYKuaWs996E4jg4pump,"3,52",NaN,326,220,106,220,106,"200723,22","200723,22",...,NaN,0,18800.0 18100.0 0.23 23500.0 0.1 0.1 23.0 7.05...,0 0 0 0 0 0 18.0 5.28 15.0 0 0 10.0 0 0 0 0 0 ...,76.0,3.0,21.0,0 0 0 0 266.0 0 0 163.0 0 0 674.0 0 0 0 0 206....,12800.0 11000.0 10200.0 10100.0 9213.0 7649.0 ...,"72,69"
5199,7ukS3hzNBqaZ9jxkUMJze7DUM8mskdDEFoYeUBe1pump,"3,34",NaN,538,288,250,288,250,"221432,61","221432,61",...,NaN,0,235.0 1177.0 228.0 4667.0 0.1 350.0 701.0 11.0...,0 1411.0 493.0 6593.0 0 401.0 417.0 0 451.0 39...,26.0,21.0,53.0,0 4667.0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,1929.0 6593.0 1902.0 1420.0 1340.0 1230.0 1175...,"68,82"
5198,AAV7FB5arjZFGeMDSToXQ9KpSd3fi8m2Bcib4mazpump,"3,16",NaN,285,164,121,164,121,"49426,64","49426,64",...,NaN,0,700.0 4664.0 11.0 1176.0 35.0 0.1 0.1 0.23 11....,1179.0 5046.0 8.76 0 0 0 0 0 10.0 1.71 0 48.0 ...,35.0,33.0,32.0,700.0 1412.0 4664.0 1412.0 1882.0 0 1176.0 188...,1179.0 1822.0 5046.0 1658.0 2086.0 190.0 1364....,"68,76"


In [20]:
#df = df.dropna(subset=['transfers', 'PNL'])
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2817 entries, 5203 to 1929
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   token_address            2817 non-null   object 
 1   token_age_b              2817 non-null   object 
 2   transfers                2817 non-null   float64
 3   transactions             2817 non-null   int64  
 4   buys_m5                  2817 non-null   int64  
 5   sells_m5                 2817 non-null   int64  
 6   buys_h1                  2817 non-null   int64  
 7   sells_h1                 2817 non-null   int64  
 8   volume_m5                2817 non-null   object 
 9   volume_h1                2817 non-null   object 
 10  price_change_m5          2817 non-null   object 
 11  price_change_h1          2817 non-null   object 
 12  price_change_h6          2817 non-null   object 
 13  price_change_h24         2817 non-null   object 
 14  liquidity                2

In [21]:
float_types = [
    'token_age_b', 
    'transfers', 'transactions', 'buys_m5', 'sells_m5',
    'buys_h1', 'sells_h1', 
    'volume_m5', 'volume_h1', 'price_change_m5',
    'price_change_h1', 'price_change_h6', 'price_change_h24', 'liquidity',
    'market_cap', 'PNL',
],

int_types = [
    'is_mutable_metadata', 'is_telegram', 'is_twitter',
    'is_website', 'boosts', 'telegram_members', 'is_telegram_error',
    'twitter_days', 'twitter_tweets', 'twitter_followers',
    'twitter_smart_followers', 'is_twitter_error',
    'sns_held_all', 'sns_sold_some', 'sns_sold_all',
]

for col in float_types:
    df[col] = df[col].replace(",",".", regex=True).astype(float)
    
for col in int_types:
    df[col] = df[col].replace(",",".", regex=True).astype(int)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2817 entries, 5203 to 1929
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   token_address            2817 non-null   object 
 1   token_age_b              2817 non-null   float64
 2   transfers                2817 non-null   float64
 3   transactions             2817 non-null   float64
 4   buys_m5                  2817 non-null   float64
 5   sells_m5                 2817 non-null   float64
 6   buys_h1                  2817 non-null   float64
 7   sells_h1                 2817 non-null   float64
 8   volume_m5                2817 non-null   float64
 9   volume_h1                2817 non-null   float64
 10  price_change_m5          2817 non-null   float64
 11  price_change_h1          2817 non-null   float64
 12  price_change_h6          2817 non-null   float64
 13  price_change_h24         2817 non-null   float64
 14  liquidity                2

In [23]:
#df = df[df['is_telegram_error'] == 0]
#df = df[df['is_twitter_error'] == 0]
#df = df[df['price_change_m5'] > -10]
#df = df[df['price_change_h1'] > 10]
#df = df[(df['transfers'] > 100) & (df['transfers'] < 5000)]

In [24]:
df = df.drop(['is_telegram_error', 'is_twitter_error'], axis=1)

In [25]:
df.loc[df["PNL"] >= 60, "profit"] = 1
df.loc[df["PNL"] < 60, "profit"] = 0
#df = df.drop(["PNL"], axis = 1)
df["profit"] = df["profit"].astype(int)

In [26]:
df = df.dropna(subset=['sns_bought', 'sns_sold', 'tt_bought', 'tt_sold'])

In [27]:
df = df[df['sns_bought'] != 0]
df = df[df['sns_sold'] != 0]
df = df[df['tt_bought'] != 0]
df = df[df['tt_sold'] != 0]

In [28]:
def covert_to_list(df, col_name):
    all_lsts = []
    for value_str in df[col_name]:
        lst = [float(x) for x in value_str.split(" ")]
        all_lsts.append(lst)
        
    return all_lsts

In [29]:
df['sns_bought_lst'] = covert_to_list(df, 'sns_bought')
df['sns_sold_lst'] = covert_to_list(df, 'sns_sold')
df['tt_bought_lst'] = covert_to_list(df, 'tt_bought')
df['tt_sold_lst'] = covert_to_list(df, 'tt_sold')

In [30]:
from collections import Counter

def get_most_common(df, col_name, is_zero=True):
    result = []
    for lst in df[col_name]:
        copy_lst = lst.copy()
        if not is_zero:
            if 0.0 in copy_lst:
                copy_lst = list(
                    filter(
                        lambda x: x != 0.0,
                        lst
                    )
                )
        if copy_lst:
            occurence_count = Counter(copy_lst)
            result.append(occurence_count.most_common(1)[0])
        else:
            result.append((0.0, 0.0))
        #print(occurence_count.most_common(1)[0][0])
    return result
    
df['sns_most_common'] = [x[0] for x in get_most_common(df, 'sns_bought_lst')]
df['sns_most_common_except_0'] = [x[0] for x in get_most_common(df, 'sns_bought_lst', False)]

In [31]:
def sum_values(df: pd.DataFrame, col_name: str, num: int) -> pd.DataFrame:
    new_values_lst = []
    for lst in df[col_name]:
        end = len(lst) + 1 if len(lst) < num else num
        result = 0
        for value in lst[:end]:
            result += value
        new_values_lst.append(result)
        
    return new_values_lst

In [32]:
df['tt_f10_bought']= sum_values(df, 'tt_bought_lst', 100)
df['tt_f10_sold'] = sum_values(df, 'tt_sold_lst', 100)
df['sns_f10_bought'] = sum_values(df, 'sns_bought_lst', 100)
df['sns_f10_sold'] = sum_values(df, 'sns_sold_lst', 100)

In [33]:
def get_pnl(df: pd.DataFrame, bought_col: str, sold_col: str) -> list[list]:
    all_bought = []
    for value_str in df[bought_col]:
        bought_lst = [float(x) for x in value_str.split(" ")]
        all_bought.append(bought_lst)
        
    all_sold = []
    for value_str in df[sold_col]:
        sold_lst = [float(x) for x in value_str.split(" ")]
        all_sold.append(sold_lst)
    
    all_pnl = []
    for each_bought, each_sold in zip(all_bought, all_sold):
        each_pnl = [sold - bought if sold else 0 for bought, sold in zip(each_bought, each_sold)]
        all_pnl.append(each_pnl)
    
    return all_pnl

In [34]:
df['sns_pnl'] = get_pnl(df, 'sns_bought', 'sns_sold')
df['tt_pnl'] = get_pnl(df, 'tt_bought', 'tt_sold')

In [35]:
def count_zero(df: pd.DataFrame, col_name: str, amount: int | None=None):
    count_zero = []
    for lst in df[col_name]:
        count_zero.append(lst.count(0))
    
    return count_zero

In [36]:
df['sns_no_bought'] = count_zero(df, 'sns_bought_lst')
df['sns_no_sold'] = count_zero(df, 'sns_sold_lst')
df['tt_no_bought'] = count_zero(df, 'tt_bought_lst')
df['tt_no_sold'] = count_zero(df, 'tt_sold_lst')

In [37]:
def count_pnl_loss(df: pd.DataFrame, col_name: str):
    count_pnl_loss = []
    for pnl_lst in df[col_name]:
        pnl_loss = sum(i < 0 for i in pnl_lst)
        count_pnl_loss.append(pnl_loss)
        
    return count_pnl_loss

In [38]:
df['sns_pnl_loss'] = count_pnl_loss(df, 'sns_pnl')
df['tt_pnl_loss'] = count_pnl_loss(df, 'tt_pnl')

In [39]:
df = df.drop(
    [
        'tt_bought', 'tt_sold', 
        'sns_bought', 'sns_sold', 
        'tt_bought_lst', 'tt_sold_lst', 
        'sns_bought_lst', 'sns_sold_lst', 
        'sns_pnl', 'tt_pnl', 
    ], 
    axis=1
)

In [40]:
def get_pairs_count(token_address):
    time.sleep(1)
    try:
        pairs = requests.get("https://api.dexscreener.com/latest/dex/tokens/" + token_address).json()["pairs"]
        count = len(pairs)
    except:
        count = 0
    return count

In [41]:
df.columns

Index(['token_address', 'token_age_b', 'transfers', 'transactions', 'buys_m5',
       'sells_m5', 'buys_h1', 'sells_h1', 'volume_m5', 'volume_h1',
       'price_change_m5', 'price_change_h1', 'price_change_h6',
       'price_change_h24', 'liquidity', 'market_cap', 'is_mutable_metadata',
       'is_telegram', 'is_twitter', 'is_website', 'boosts', 'telegram_members',
       'twitter_days', 'twitter_tweets', 'twitter_followers',
       'twitter_smart_followers', 'sns_held_all', 'sns_sold_some',
       'sns_sold_all', 'PNL', 'profit', 'sns_most_common',
       'sns_most_common_except_0', 'tt_f10_bought', 'tt_f10_sold',
       'sns_f10_bought', 'sns_f10_sold', 'sns_no_bought', 'sns_no_sold',
       'tt_no_bought', 'tt_no_sold', 'sns_pnl_loss', 'tt_pnl_loss'],
      dtype='object')

In [42]:
# df["pairs_count"] = df["token_address"].apply(get_pairs_count)

In [43]:
df = df.drop(["token_address"], axis=1)

In [44]:
df.to_csv('./clean_data.csv', index=False)

In [45]:
df.to_excel('./clean_data.xlsx', index=False)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2807 entries, 5203 to 1929
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   token_age_b               2807 non-null   float64
 1   transfers                 2807 non-null   float64
 2   transactions              2807 non-null   float64
 3   buys_m5                   2807 non-null   float64
 4   sells_m5                  2807 non-null   float64
 5   buys_h1                   2807 non-null   float64
 6   sells_h1                  2807 non-null   float64
 7   volume_m5                 2807 non-null   float64
 8   volume_h1                 2807 non-null   float64
 9   price_change_m5           2807 non-null   float64
 10  price_change_h1           2807 non-null   float64
 11  price_change_h6           2807 non-null   float64
 12  price_change_h24          2807 non-null   float64
 13  liquidity                 2807 non-null   float64
 14  market_cap